In [1]:
!pip install gradio faker openai pandas matplotlib seaborn

In [2]:
import gradio as gr
import pandas as pd
from faker import Faker
import random
import openai
import re  # Para extraer números de la respuesta
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os
from dotenv import load_dotenv
import openai

# Cargar variables desde el archivo .env
load_dotenv()

# Obtener la clave API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")

# Configurar el cliente OpenAI con la API key cargada
client = openai.OpenAI(api_key=api_key)

# Verificar si la API Key se cargó correctamente
if not api_key or api_key.startswith("your-api-key"):
    raise ValueError("❌ Error: La clave API de OpenAI no se cargó correctamente.")


In [4]:
import gradio as gr
import pandas as pd
from faker import Faker
import random
import openai
import re  # Para extraer números de la respuesta
import os  # Para verificar si el archivo se guarda

# Configurar la API de OpenAI (Reemplaza con tu clave real)
openai.api_key = "your-api-key"
client = openai.Client()  # Nuevo cliente de OpenAI

# Crear la instancia de Faker para generar datos aleatorios
fake = Faker()

# Función para generar datos sintéticos y guardarlos en CSV
def generate_and_save_data(num_clients=10):
    data = []
    for _ in range(num_clients):
        name = fake.name()
        email = fake.email()
        age = random.randint(18, 80)
        country = fake.country()
        income = round(random.uniform(20000, 120000), 2)

        # Llamar a la API de OpenAI para generar el gasto mensual estimado
        prompt = f"Dado que un cliente tiene un ingreso anual de ${income}, ¿cuál sería su gasto mensual estimado? Responde con un solo número en dólares."
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente financiero que estima gastos mensuales. Solo responde con el número sin texto adicional."},
                {"role": "user", "content": prompt}
            ]
        )
        response_text = response.choices[0].message.content.strip()

        # Extraer solo el número de la respuesta usando regex
        match = re.search(r"\d+\.?\d*", response_text)
        monthly_spending = float(match.group()) if match else 0.0  # Convertir a float si es posible

        data.append({
            "Name": name,
            "Email": email,
            "Age": age,
            "Country": country,
            "Annual Income ($)": income,
            "Estimated Monthly Spending ($)": monthly_spending
        })

    # Convertir la lista en DataFrame
    df = pd.DataFrame(data)

    # Guardar en un archivo CSV
    csv_path = "customer_data.csv"
    df.to_csv(csv_path, index=False)

    return df, csv_path  # Devolvemos el DataFrame y el archivo CSV para descarga

# Función para generar datos y devolver el archivo CSV
def generate_data_gradio(num_clients):
    df, csv_path = generate_and_save_data(num_clients)
    return df, csv_path  # Retorna el DataFrame y la ruta del CSV

# Crear la interfaz con Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🏦 Synthetic Customer Data Generator")
    gr.Markdown("Genera datos de clientes ficticios y descárgalos en formato CSV.")

    num_clients = gr.Slider(5, 100, step=5, label="Número de Clientes", value=10)

    generate_button = gr.Button("Generar y Guardar CSV")
    output_data = gr.Dataframe()
    download_csv = gr.File(label="Descargar CSV")

    generate_button.click(
        fn=generate_data_gradio,
        inputs=[num_clients],
        outputs=[output_data, download_csv]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
# Cargar el archivo CSV
df = pd.read_csv("customer_data.csv")

# Mostrar las primeras filas del archivo CSV
df.head()


,Name,Email,Age,Country,Annual Income ($),Estimated Monthly Spending ($)
0,Krista Smith,tommy64@example.net,66,Poland,113787.50,9482.290000
1,Peter Martinez,xbrewer@example.net,58,Italy,109791.35,9149.280000
2,Stephanie Hernandez,laurenmorgan@example.com,23,Italy,77569.25,6464.104167
3,Amanda Dillon,jennifermartinez@example.org,53,Colombia,75232.23,6269.350000
4,Jody Garcia,ericmccall@example.com,33,Mozambique,62026.47,5172.210000


In [8]:
df.describe()

,Age,Annual Income ($),Estimated Monthly Spending ($)
count,15.000000,15.000000,15.000000
mean,44.466667,91388.902667,6653.592278
std,14.470989,17461.949502,2840.250453
min,23.000000,62026.470000,6.000000
25%,32.500000,77764.010000,6366.727083
50%,45.000000,88712.100000,7276.980000
75%,55.500000,110851.365000,8398.167500
max,67.000000,114486.460000,9540.540000


In [9]:
%matplotlib inline
import matplotlib.pyplot as plt

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

# Función para guardar la gráfica como imagen
def save_plot_as_image(df):
    # Guardar histograma del ingreso anual
    plt.figure(figsize=(8, 5))
    sns.histplot(df["Annual Income ($)"], bins=20, kde=True)
    plt.title("Distribución del Ingreso Anual")
    plt.xlabel("Ingreso Anual ($)")
    plt.ylabel("Frecuencia")
    plt.savefig("income_distribution.png")  # Guardar imagen
    plt.close()

    # Guardar scatterplot de ingreso vs gasto mensual
    plt.figure(figsize=(8, 5))
    sns.scatterplot(x=df["Annual Income ($)"], y=df["Estimated Monthly Spending ($)"])
    plt.title("Ingreso Anual vs. Gasto Mensual Estimado")
    plt.xlabel("Ingreso Anual ($)")
    plt.ylabel("Gasto Mensual Estimado ($)")
    plt.savefig("income_vs_spending.png")  # Guardar imagen
    plt.close()

    # Guardar boxplot del gasto mensual por grupo de edad
    plt.figure(figsize=(10, 6))
    df["Age Group"] = pd.cut(df["Age"], bins=[18, 30, 45, 60, 80], labels=["18-30", "31-45", "46-60", "61-80"])
    sns.boxplot(x="Age Group", y="Estimated Monthly Spending ($)", data=df)
    plt.title("Distribución del Gasto Mensual por Grupo de Edad")
    plt.xlabel("Grupo de Edad")
    plt.ylabel("Gasto Mensual Estimado ($)")
    plt.savefig("spending_by_age.png")  # Guardar imagen
    plt.close()

# Llamar a la función para guardar las imágenes
save_plot_as_image(df)

print("✅ Gráficas guardadas como imágenes.")


✅ Gráficas guardadas como imágenes.


In [33]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    

img1 = encode_image(r"C:\Users\quesa\Documents\GitHub\LLM-Based-Tutor\income_distribution.png")
img2 = encode_image(r"C:\Users\quesa\Documents\GitHub\LLM-Based-Tutor\income_vs_spending.png")
img3 = encode_image(r"C:\Users\quesa\Documents\GitHub\LLM-Based-Tutor\spending_by_age.png")


In [32]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Explica las 3 gráficas que encontrarás en las siguientes imágenes.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{img1}"},
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{img2}"},
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{img3}"},
                },
            ],
        }
    ],
)
print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Aquí tienes una explicación de las tres gráficas presentadas:\n\n### 1. Gráfica de Distribución del Ingreso Anual\n- **Tipo**: Histograma con curva de densidad.\n- **Descripción**: Esta gráfica muestra la frecuencia de los diferentes rangos de ingreso anual. Las barras representan el número de individuos (frecuencia) en cada rango de ingreso, mientras que la línea azul superpuesta indica la densidad de probabilidad. Se observa una distribución que parece tener picos en ciertos rangos (por ejemplo, alrededor de 80,000 y 110,000), lo que sugiere que hay más personas con esos ingresos anuales.\n\n### 2. Gráfica de Ingreso Anual vs. Gasto Mensual Estimado\n- **Tipo**: Gráfica de dispersión.\n- **Descripción**: Esta gráfica muestra la relación entre el ingreso anual y el gasto mensual estimado. Cada punto representa a una persona o unidad de análisis, con el ingreso anual en el eje x y el gasto mensu